# Ch. 3 Exercises
<ol>
<li>[Kalman Car](#Kalman_Car)</li>
<li>[Measurement](#Measurement)</li>
<li>[Derivation](#Derivation)</li>
<li>[EKF Linearization](#EKF_Linearization)</li>
</ol>

# 1. Kalman Car <a id="Kalman_Car"></a>
\begin{equation}
x_{t} = A_{t}x_{t-1} + B_{t}u_{t} + \varepsilon_{t} \\
z_{t} = C_{t}x_{t} + \delta_{t}
\end{equation}

\begin{equation}
Kalman\_Filter(\mu_{t-1}, \Sigma_{t-1}, u_{t}, z_{t}) \\
\bar{\mu}_{t} = A_{t}\mu_{t-1} + B_{t}u_{t} \\
\bar{\Sigma}_{t} = A_{t}\Sigma_{t-1}A^{T}_{t} + R_{t}\\
K_{t} = \bar{\Sigma}_{t}C^{T}_{t}(C_{t}\bar{\Sigma}_{t}C^{T}_{t} + Q_{t})^{-1} \\
\mu_{t} =  \bar{\mu}_{t} + K_{t}(z_{t}-C_{t}\bar{\mu_{t}}) \\
\Sigma_{t} = (I - K_{t}C_{t})\bar{\Sigma_{t}}
\end{equation}

(a) Minimum State Vector:
\begin{equation}
\mu = \left[\begin{array}{c}
x \\
\dot{x} \end{array}\right]
\end{equation}

(b) State Transition Matricies

\begin{equation}
A = \left[\begin{array}{cc}
1 & 1 \\
0 & 1 \end{array}\right],
\space
B = \left[\begin{array}{c}
0 \\
0 \end{array}\right],
\space
R = \left[\begin{array}{cc}
\frac{1}{4} & \frac{1}{2} \\
\frac{1}{2} & 1 \end{array}\right]
\end{equation}


In [ ]:

#(b) State Transition Matricies
import numpy as np
from copy import copy
A =  np.array([[1, 1], [0, 1]])
B = np.array([[0], [0]])
# Noise for acceleration directly varies the velocity, but only affects the position with a factor of 1/2
# v = a*t
# d = v*t + 1/2 a*t**2
# variance (V) = [0.5, 1]. R = V * V.T
R = np.array([[0.25, 0.5], [0.5, 1.0]])
#
# p(xt|ut,xt_1) = det(2*pi*Rt)**-0.5 * exp(-0.5*(xt-A*xt_1 - B*ut).T*inv(R)*(xt-A*xt_1*B*ut)
#
#(c) State Prediction
def state_prediction(mu_t_1, sigma_t_1, u_t):
    mu_bar_t = A.dot(mu_t_1) + B.dot(u_t)
    sigma_bar_t = A.dot(sigma_t_1).dot(A.T) + R
    
    return (mu_bar_t, sigma_bar_t)

mu = np.array([[0], [0]])
sigma = np.array([[0, 0], [0, 0]])
u = np.array([[0]])

for i in range(6):
    print('t = %d' % (i))
    print('\t'+'mu = \n\t\t%s' % str(mu).replace('\n','\n\t\t'))
    print('\t'+'sigma = \n\t\t%s' % str(sigma).replace('\n','\n\t\t'))
    print('')
    mu, sigma = state_prediction(mu, sigma, u)

In [ ]:
#(d) Plot the posterior
import pylab
%pylab inline

from matplotlib.patches import Ellipse

def generate_ellipse(mu, sigma):
    eigen_values, eigen_vectors = np.linalg.eig(sigma)
    max_e_index = eigen_values.argmax()
    min_e_index = eigen_values.argmin()
    max_e_vect = eigen_vectors[:,max_e_index]
    angle = np.arctan2(max_e_vect[1], max_e_vect[0])
    return Ellipse( xy=mu.flatten(), 
                    width=np.sqrt(eigen_values[max_e_index]), 
                    height=np.sqrt(eigen_values[min_e_index]), 
                    angle=angle * 180.0/np.pi,
                    fill=False)

mu = np.array([[0], [0]])
sigma = np.array([[0, 0], [0, 0]])

ellipses = []
for i in range(6):
    ellipses.append(generate_ellipse(mu, sigma))
    mu, sigma = state_prediction(mu, sigma, [0])

fig = pylab.figure()
ax = fig.add_subplot(111, aspect='equal')
for e in ellipses:
    ax.add_artist(e)
    e.set_clip_box(ax.bbox)
    #e.set_alpha(rand())
    #e.set_facecolor(rand(3))
    
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_xlabel('$x$')
ax.set_ylabel('$\dot{x}$')
fig.suptitle('1 $\sigma$ uncertianty ellipse')


pylab.show()

(f) As $t\to\infty$ the ellipse's major axes will be parallel with the x axis, meaning that the current velocity gives no extra information about the position


## 2. Measurement <a id="Measurement"></a>
We can measure the position with an uncertainty of $\sigma^2 = 10$

(a) Measurement Matricies
\begin{equation}
C = \left[\begin{array}{cc}
1 & 0 \end{array}\right],
\space
Q = \left[\begin{array}{c}
10 \end{array}\right],
\end{equation}

(b) Measurement Update


In [ ]:
c = np.array([[1,0]])
q = np.array([[10]])
def measurement_update(mu_bar_t, sigma_bar_t, z_t):
    k_t = sigma_bar_t.dot(c.T).dot(np.linalg.inv(c.dot(sigma_bar_t).dot(c.T) + q))
    mu_t = mu_bar_t + k_t.dot(z_t - c.dot(mu_bar_t))
    sigma_t = (np.eye(k_t.shape[0]) - k_t.dot(c)).dot(sigma_bar_t)
    
    return mu_t, sigma_t

In [ ]:
mu = np.array([[0], [0]])
sigma = np.array([[0, 0], [0, 0]])
u = np.array([[0]])

for i in range(5):
    mu, sigma = state_prediction(mu, sigma, u)
    
before_ellipse = generate_ellipse(mu, sigma)

mu, sigma = measurement_update(mu, sigma, np.array([[5]]))

after_ellipse = generate_ellipse(mu, sigma)

fig = pylab.figure()
ax = fig.add_subplot(111, aspect='equal')
for e in (before_ellipse, after_ellipse):
    ax.add_artist(e)
    e.set_clip_box(ax.bbox)
    #e.set_alpha(rand())
    #e.set_facecolor(rand(3))
    
ax.set_xlim(-5.5, 5.5)
ax.set_ylim(-5.5, 5.5)
ax.set_xlabel('$x$')
ax.set_ylabel('$\dot{x}$')
fig.suptitle('1 $\sigma$ uncertianty ellipse')



## 3. Derivation  <a id="Derivation"></a>

## 4. EKF Linearization <a id="EKF_Linearization"></a>

\begin{equation}
Extended\_Kalman\_Filter(\mu_{t-1}, \Sigma_{t-1}, u_{t}, z_{t}) \\
\bar{\mu}_{t} = g(u_{t}, \mu_{t-1})\\
\bar{\Sigma}_{t} = G_{t}\Sigma_{t-1}G^{T}_{t} + R_{t}\\
K_{t} = \bar{\Sigma}_{t}H^{T}_{t}(H_{t}\bar{\Sigma}_{t}H^{T}_{t} + Q_{t})^{-1} \\
\mu_{t} =  \bar{\mu}_{t} + K_{t}(z_{t}-h(\bar{\mu_{t}})) \\
\Sigma_{t} = (I - K_{t}H_{t})\bar{\Sigma_{t}}
\end{equation}

We have have a robot whose location is well known, but it's orientation is unknown
\begin{equation}
state = \left[\begin{array}{ccc}
x \\ y \\ \sigma \end{array}\right]
\end{equation}

\begin{equation}
\mu = \left[\begin{array}{c}
0 \\ 0 \\ 0 \end{array}\right],
\space
\sigma = \left[\begin{array}{ccc}
0.01 & 0 & 0 \\
0 & 0.01 & 0 \\
0 & 0 & 10000 \end{array}\right]
\end{equation}

The expected location after the robot moves $d = 1$ unit forward is given by

\begin{equation}
\left[\begin{array}{c}
x^{\prime} \\ y^{\prime} \\ \theta^{\prime} \end{array}\right]
 = \left[\begin{array}{c}
x + cos(\theta) \\ y + sin(\theta) \\ \theta \end{array}\right]
\end{equation}

(a) Draw your best model of the posterior over the robot pose in x and y.

I have no knowledge of the heading so I'm going to guess that it will be within a circle of radius 1, plus or minus the uncertintanty of 0.01

In [ ]:
my_model_inner = Ellipse( xy=[0,0], width=2*1-0.01, height=2*1-0.01, angle=0, fill=False)
my_model_outer = Ellipse( xy=[0,0], width=2*1+0.01, height=2*1+0.01, angle=0, fill=False)

fig = pylab.figure()
ax = fig.add_subplot(111, aspect='equal')
for e in (my_model_inner, my_model_outer):
    ax.add_artist(e)
    e.set_clip_box(ax.bbox)
    
ax.set_xlim(-1.1, 1.1); ax.set_ylim(-1.1, 1.1)
ax.set_xlabel('$x$'); ax.set_ylabel('$y$')
fig.suptitle('1 $\sigma$ uncertianty ellipse')

(b) EKF Prediction

Since the state does not depend on time the A matrix is the identity. 
\begin{equation}
A = \left[\begin{array}{ccc}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \end{array}\right]
\end{equation}

The nonlinear update equation g is given by 
\begin{equation}
x_{t-1} = \left[\begin{array}{c}
x_{1,t-1} \\
x_{2,t-1} \\
x_{3,t-1} \end{array}\right],
\space
g(u_{t}, x_{t-1}) = \left[\begin{array}{c}
x_{1,t-1} + u_{t}*cos(x_{3,t-1}) \\
x_{2,t-1} + u_{t}*sin(x_{3,t-1}) \\
x_{3,t-1} \end{array}\right]
\end{equation}

The definition of the Jacobian (from Wikipedia)
\begin{equation}
J = \frac{d\mathbf f}{d\mathbf x} = \begin{bmatrix}
    \dfrac{\partial \mathbf{f}}{\partial x_1}; \cdots; \dfrac{\partial \mathbf{f}}{\partial x_n} \end{bmatrix}
= \begin{bmatrix}
    \dfrac{\partial f_1}{\partial x_1}; \cdots; \dfrac{\partial f_1}{\partial x_n}\\
    \vdots; \ddots; \vdots\\
    \dfrac{\partial f_m}{\partial x_1}; \cdots; \dfrac{\partial f_m}{\partial x_n} \end{bmatrix}
\end{equation}

The partial derivitive (Jacobain) $g^{\prime}(u_{t}, x_{t-1})=\frac{\delta g(u_{t}, x_{t-1})}{\delta x_{t-1}}$ is given by

\begin{equation}
g^{\prime}(u_{t}, x_{t-1}) = \left[\begin{array}{ccc}
1 & 0 & u_{t}*sin(x_{3,t-1}) \\
0 & 1 & -u_{t}*cos(x_{3,t-1}) \\
0 & 0 & 1 \end{array}\right]
\end{equation}

Linearizing at $t=0$ gives
\begin{equation}
G_{0} = g^{\prime}(u_{t}, x_{t-1}) =
\left[\begin{array}{ccc}
1 & 0 & 1*sin(0) \\
0 & 1 & -1*cos(0) \\
0 & 0 & 1 \end{array}\right] =
\left[\begin{array}{ccc}
1 & 0 & 0 \\
0 & 1 & -1 \\
0 & 0 & 1 \end{array}\right]
\end{equation}





In [ ]:
mu_t = np.array([[0], [0], [0]])
sigma_t = np.array([[0.01, 0, 0], [0, 0.01, 0], [0, 0, 10000]])
u_t = np.array([[1]])
R_t = np.array([[0], [0], [0]])

def g(u_t, mu_t_1):
    """Returns mu_bar_t"""
    return mu_t_1 + np.array([[u_t[0,0]*np.cos(mu_t_1[0,0])],
                              [u_t[0,0]*np.sin(mu_t_1[1,0])],
                              [0]])

def G(u_t, mu_t_1):
    """Returns G_t"""
    return np.array([[1, 0, u_t[0,0]*np.sin(mu_t_1[2,0])], 
                     [0, 1, -u_t[0,0]*np.cos(mu_t_1[2,0])], 
                     [0, 0, 1]])

In [ ]:
mu_bar_t = g(u_t, mu_t)
G_t = G(u_t, mu_t)
sigma_bar_t = G_t.dot(sigma_t).dot(G_t.T)+R_t
G_t

In [ ]:
#(c) Draw the uncertainty ellipse

#get just the x and y state variables
mu_xy = mu_bar_t[:2,:]
sigma_xy = sigma_bar_t[:2,:2]

uncertainty_e = generate_ellipse(mu_xy, sigma_xy)

my_model_inner = Ellipse( xy=[0,0], width=2*1-0.01, height=2*1-0.01, angle=0, fill=False)
my_model_outer = Ellipse( xy=[0,0], width=2*1+0.01, height=2*1+0.01, angle=0, fill=False)

fig = pylab.figure()
ax1 = fig.add_subplot(111, aspect='equal')
for e in (my_model_inner, my_model_outer, uncertainty_e):
    ax1.add_artist(e)
    e.set_clip_box(ax1.bbox)  
ax1.set_xlim(-55, 55); ax1.set_ylim(-55, 55)
ax1.set_xlabel('$x$'); ax1.set_ylabel('$y$')

fig.suptitle('1 $\sigma$ uncertianty ellipse')

(d) Noisy measurement with $Q = 0.01$ of the robot's x position

The measurement model 
\begin{equation}
H = \left[\begin{array}{ccc}
1 & 0 & 0 \end{array}\right],
\space
Q = \left[\begin{array}{c}
0.01 \end{array}\right],
\end{equation}


In [ ]:
H = np.array([[1, 0, 0]])
Q = np.array([[0.01]])

def ekf_measurement_update(mu_bar_t, sigma_bar_t, z_t):
    k_t = sigma_bar_t.dot(H.T).dot(np.linalg.inv(H.dot(sigma_bar_t).dot(H.T) + Q))
    mu_t = mu_bar_t + k_t.dot(z_t - H.dot(mu_bar_t))
    sigma_t = (np.eye(k_t.shape[0]) - k_t.dot(H)).dot(sigma_bar_t)
    return mu_t, sigma_t

z_t = np.array([[0]])

mu_t, sigma_t = ekf_measurement_update(mu_bar_t, sigma_bar_t, z_t)


#get just the x and y state variables
mu_xy = mu_t[:2,:]
sigma_xy = sigma_t[:2,:2]

uncertainty_e = generate_ellipse(mu_xy, sigma_xy)
my_model_inner = Ellipse( xy=[0,0], width=2*1-0.01, height=2*1-0.01, angle=0, fill=False)
my_model_outer = Ellipse( xy=[0,0], width=2*1+0.01, height=2*1+0.01, angle=0, fill=False)

fig = pylab.figure()
ax1 = fig.add_subplot(111, aspect='equal')
for e in (my_model_inner, my_model_outer, uncertainty_e):
    ax1.add_artist(e)
    e.set_clip_box(ax1.bbox)  
ax1.set_xlim(-55, 55); ax1.set_ylim(-55, 55)
ax1.set_xlabel('$x$'); ax1.set_ylabel('$y$')

fig.suptitle('1 $\sigma$ uncertianty ellipse')


The measurement moves the uncertainty ellipse along the $x$ axis to half way between the measurement and the prior estimation of $x=1$.
Intutiviely, if we got the x measurement, we would be able to limit the y uncertainty to two locations.

(e) The differences are rather significant between the EKF and my intutive results. The results could be made more accurate by reducing the uncertainty of the angle and increasing the uncertainty of the position. This would prevent the results from getting so distorted.

If the angle was known, but the y coordinate was not known, the ellipse would look similar to after the linearization, but would track the robot more accurately.